# Stage 2 위조 비율 실험: MIL Bag Generation (5/10/20/30/50% Forgery) - L=20 기준

이 노트북은 Stage 1에서 추출한 ArcFace 임베딩을 이용해 Multiple Instance Learning(MIL) 학습을 위한 **다양한 위조 비율** Bag 데이터를 생성합니다.

**핵심 개선 (L=20 기준):**
- **정확한 위조 비율**: 5%, 10%, 20%, 30%, 50%로 양성 bag 내 B 작성자 비율을 L=20 토큰 기준으로 정확히 조절
- **FCR/IER 이중 분석**: 토큰 기준(FCR)과 인스턴스 기준(IER) 위조 비율을 동시 측정
- **두 가지 실험 모드**: Matched (각 비율별 독립 학습) vs Shift (30% 학습, 다양한 비율 평가)
- **고정 윈도우**: (win=5, stride=1) → 16개 가능 인스턴스 중 10개 사용 → 각 bag은 (10, 5, 256)
- **50/50 균형**: 각 split에서 정확히 50% Positive, 50% Negative 비율

**향상된 메타데이터:**
- **FCR (Forgery Content Ratio)**: 문서 내 실제 위조 토큰 비율
- **IER (Instance Exposure Ratio)**: 모델이 체감하는 위조 인스턴스 비율
- **b_writer**: 위조자(B 작성자) 식별 정보
- **기대값 vs 실측값**: 이론적 기대치와 실제 생성값 비교

In [1]:
# 라이브러리 임포트 및 환경 설정
import os, random, pickle, numpy as np, pandas as pd
from collections import defaultdict

# GPU 설정 (선택적)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = os.getenv('MIL_STAGE2_GPU', '3')

# 재현성을 위한 시드 설정
SEED_BASE = 42
np.random.seed(SEED_BASE)
random.seed(SEED_BASE)

print(f'환경 설정 완료: GPU={os.environ.get("CUDA_VISIBLE_DEVICES", "N/A")}, SEED={SEED_BASE}')

환경 설정 완료: GPU=3, SEED=42


In [2]:
# 경로 설정 및 데이터 로딩
embedding_dir = '/workspace/MIL/data/processed/embeddings'
raw_meta_csv  = '/workspace/MIL/data/raw/naver_ocr.csv'  # 선택 사항
bags_dir      = '/workspace/MIL/data/processed/bags'
os.makedirs(bags_dir, exist_ok=True)

margin_value = '0.4'
rng_global = np.random.default_rng(SEED_BASE)

# CSV 로딩 함수
def load_split_csv(split):
    csv_path = os.path.join(embedding_dir, f'mil_arcface_margin_{margin_value}_{split}_data.csv')
    df = pd.read_csv(csv_path)
    label_col = 'label' if 'label' in df.columns else 'author_id'
    emb_cols  = [c for c in df.columns if c.startswith('embedding')]
    assert len(emb_cols) > 0, "No embedding_* columns found."
    return df, label_col, emb_cols

# 데이터 로딩
train_df, label_col, emb_cols = load_split_csv('train')
val_df,   _,         _        = load_split_csv('val')
test_df,  _,         _        = load_split_csv('test')
embed_dim = len(emb_cols)

print(f"데이터 로딩 완료:")
print(f"  - Embedding 차원: {embed_dim}")
print(f"  - Label 컬럼: {label_col}")
print(f"  - Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# 선택적 원본 메타데이터 로딩
try:
    original_df = pd.read_csv(raw_meta_csv)
    print(f"  - 원본 메타데이터: {len(original_df)} rows")
    has_raw_meta = True
except Exception:
    print(f"  - 원본 메타데이터: 없음")
    has_raw_meta = False

데이터 로딩 완료:
  - Embedding 차원: 256
  - Label 컬럼: label
  - Train: 208233, Val: 70533, Test: 72457
  - 원본 메타데이터: 556128 rows


In [3]:
# ==============================================================
# Stage 2 위조 비율 실험: MIL Bags (5/10/20/30% Forgery)
# ==============================================================

# 작성자별 인덱스 구축
def build_writer_index(df, label_col, emb_cols):
    w2 = {}
    for wid, g in df.groupby(label_col):
        w2[int(wid)] = {
            'emb': g[emb_cols].to_numpy(dtype=np.float32),
            'paths': g['path'].tolist() if 'path' in g.columns else [''] * len(g),
            'idx': g.index.to_list()
        }
    return w2

train_writers = build_writer_index(train_df, label_col, emb_cols)
val_writers   = build_writer_index(val_df,   label_col, emb_cols)
test_writers  = build_writer_index(test_df,  label_col, emb_cols)

def list_writer_ids(wdict): 
    return list(wdict.keys())

print(f"작성자 인덱스 구축 완료:")
print(f"  - Train writers: {len(train_writers)}")
print(f"  - Val writers: {len(val_writers)}")
print(f"  - Test writers: {len(test_writers)}")

# 샘플링 헬퍼 함수들
def sample_k(n, k, rng, replace_if_needed=True):
    """n개 중에서 k개 샘플링 (부족하면 중복 허용)"""
    if (not replace_if_needed) and n >= k:
        return rng.choice(n, size=k, replace=False).tolist()
    # 부족하면 중복 허용
    return rng.choice(n, size=k, replace=True).tolist()

def sliding_windows(seq, win=5, stride=1):
    """슬라이딩 윈도우로 시퀀스 분할"""
    # seq: list of tuples (emb, wid, path, orig_idx)
    windows, metas = [], []
    for i in range(0, len(seq) - win + 1, stride):
        chunk = seq[i:i+win]
        windows.append(np.stack([e for (e,_,_,_) in chunk], axis=0))  # (5, D)
        metas.append({
            'window_idx': i,
            'word_indices': [oi for (_,_,_,oi) in chunk],
            'word_paths':   [p  for (_,_,p, _) in chunk],
            'writer_ids':   [w  for (_,w,_, _) in chunk],
        })
    return windows, metas

def pack(words, wid, W):
    """단어 인덱스를 임베딩 튜플로 변환"""
    emb, paths, idxs = W['emb'], W['paths'], W['idx']
    return [(emb[w], wid, paths[w], idxs[w]) for w in words]

print("✓ 헬퍼 함수 정의 완료")

작성자 인덱스 구축 완료:
  - Train writers: 180
  - Val writers: 60
  - Test writers: 60
✓ 헬퍼 함수 정의 완료


In [4]:
# ==== [NEW] Forgery ratio utilities ====
TOTAL_TOKENS_POS = 20        # 양성 문서 총 단어 수 (기존 14 → 20으로 증가)
TOK_NEG = 20                 # 음성 문서 총 단어 수 (기존 14 → 20으로 증가)
WIN = 5
STRIDE = 1
INSTANCES_PER_BAG = 10

def ratio_to_counts_exact(pos_ratio: float, L: int = TOTAL_TOKENS_POS):
    """
    pos_ratio(0~0.5 권장)를 토대로 B작성자 토큰 수를 정확히 계산.
    최소 1개씩은 A/B가 들어가도록 안전장치.
    예: 0.05 → B=1, A=19 / 0.30 → B=6, A=14
    """
    target = int(L * pos_ratio)  # L=20 기준 정확히 떨어짐 (0.05→1, 0.10→2, ...)
    b = max(1, min(target, L-1))  # A/B 최소 1개 보장
    a = L - b
    return a, b

def ratio_tag(pos_ratio: float) -> str:
    """파일명 태그: 0.05 → '05p', 0.30 → '30p'"""
    return f"{int(round(pos_ratio*100)):02d}p"

# IER(인스턴스 기준 위조비율) 계산 함수들
from math import comb

def expected_ier(L: int, W: int, b: int) -> float:
    """
    모든 K윈도우를 고려한 기대값(무작위 가정)
    IER = 1 - P(윈도우에 B작성자 토큰이 0개)
    """
    if b <= 0: return 0.0
    # P(윈도우에 B가 0개) = C(L-W, b) / C(L, b)
    return 1.0 - (comb(L-W, b) / comb(L, b))

def realized_ier(metas, b_writer: int, used_K: int) -> float:
    """
    실제 생성된 인스턴스에서 B작성자가 포함된 윈도우 비율
    metas: 생성된 인스턴스 메타(길이 used_K), 각 원소에 writer_ids(길이 W)
    """
    cnt = sum(1 for m in metas if b_writer in m['writer_ids'])
    return cnt / float(used_K)

print("✓ 위조 비율 유틸리티 함수 정의 완료 (L=20 기준)")
print(f"  - 총 토큰: {TOTAL_TOKENS_POS}개")
print(f"  - 윈도우: (win={WIN}, stride={STRIDE})")
print(f"  - 인스턴스/Bag: {INSTANCES_PER_BAG}개")
print(f"  - 가능한 전체 윈도우: K={(TOTAL_TOKENS_POS - WIN) + 1}개")

✓ 위조 비율 유틸리티 함수 정의 완료 (L=20 기준)
  - 총 토큰: 20개
  - 윈도우: (win=5, stride=1)
  - 인스턴스/Bag: 10개
  - 가능한 전체 윈도우: K=16개


In [5]:
# Bag 생성 함수 (위조 비율 지원 + IER 계산)

def make_negative_bag(wid, W, rng):
    """단일 작성자 Bag (레이블 0) - L=20 기준"""
    emb, paths, idxs = W['emb'], W['paths'], W['idx']
    sel = sample_k(len(emb), TOK_NEG, rng, replace_if_needed=True)
    seq = pack(sel, wid, W)
    wins, metas = sliding_windows(seq, WIN, STRIDE)
    bag = np.stack(wins[:INSTANCES_PER_BAG], axis=0)  # (10, 5, D)
    
    # 음성 bag 메타데이터 (FCR/IER 모두 0)
    ratio_meta = {
        'FCR_target': 0.0,
        'FCR_realized': 0.0,
        'tokens_A': int(TOK_NEG),
        'tokens_B': 0,
        'b_writer': None,
        'L': TOK_NEG, 'W': WIN, 'K_total': (TOK_NEG - WIN) + 1, 'instances_used': INSTANCES_PER_BAG,
        'IER_expected_allK': 0.0,
        'IER_realized_usedK': 0.0,
    }
    
    return bag, metas[:INSTANCES_PER_BAG], [int(wid)], ratio_meta

def make_positive_bag(widA, widB, WA, WB, rng, pos_ratio: float):
    """
    복수 작성자 Bag (레이블 1): A/B 토큰 수를 pos_ratio로 정확히 결정 → 전체 셔플 → 윈도우
    L=20 기준으로 FCR과 IER을 모두 계산하여 메타데이터에 기록.
    """
    # 1) A/B 토큰 수 정확 계산 (FCR)
    TOK_POS_A, TOK_POS_B = ratio_to_counts_exact(pos_ratio, TOTAL_TOKENS_POS)

    embA, pathsA, idxA = WA['emb'], WA['paths'], WA['idx']
    embB, pathsB, idxB = WB['emb'], WB['paths'], WB['idx']
    
    # 2) 샘플링 (가능한 경우 중복없이, 부족할 때만 중복 허용)
    selA = sample_k(len(embA), TOK_POS_A, rng, replace_if_needed=False)
    selB = sample_k(len(embB), TOK_POS_B, rng, replace_if_needed=False)

    seqA = [(embA[i], int(widA), pathsA[i], idxA[i]) for i in selA]
    seqB = [(embB[i], int(widB), pathsB[i], idxB[i]) for i in selB]
    
    # 3) 전체 시퀀스 셔플
    seq = seqA + seqB
    rng.shuffle(seq)

    # 4) 윈도우 생성
    wins, metas = sliding_windows(seq, WIN, STRIDE)
    bag = np.stack(wins[:INSTANCES_PER_BAG], axis=0)
    used_metas = metas[:INSTANCES_PER_BAG]

    # 5) FCR 검증 (토큰 기준)
    token_writer_ids = [w for (_,w,_,_) in seq]
    a_count = token_writer_ids.count(int(widA))
    b_count = token_writer_ids.count(int(widB))
    fcr_realized = b_count / (a_count + b_count)

    # 6) IER 계산 (인스턴스 기준)
    L = TOTAL_TOKENS_POS
    W_win = WIN
    K_total = (L - W_win) + 1
    ier_exp = expected_ier(L, W_win, TOK_POS_B)     # 기대값
    ier_real = realized_ier(used_metas, b_writer=int(widB), used_K=INSTANCES_PER_BAG)  # 실측값

    # 7) 확장된 메타데이터
    ratio_meta = {
        'FCR_target': float(pos_ratio),
        'FCR_realized': float(fcr_realized),
        'tokens_A': int(a_count),
        'tokens_B': int(b_count),
        'b_writer': int(widB),
        'L': L, 'W': W_win, 'K_total': K_total, 'instances_used': INSTANCES_PER_BAG,
        'IER_expected_allK': float(ier_exp),
        'IER_realized_usedK': float(ier_real),
    }

    return bag, used_metas, [int(widA), int(widB)], ratio_meta

print("✓ Bag 생성 함수 정의 완료 (L=20 + FCR/IER 지원)")
print(f"  - Negative: {TOK_NEG}개 단어 → {INSTANCES_PER_BAG}개 인스턴스")
print(f"  - Positive: 비율별 A+B개 단어 셔플 → {INSTANCES_PER_BAG}개 인스턴스")
print(f"  - FCR: 토큰 기준 위조 비율 (문서 현실)")
print(f"  - IER: 인스턴스 기준 위조 비율 (모델 체감)")

✓ Bag 생성 함수 정의 완료 (L=20 + FCR/IER 지원)
  - Negative: 20개 단어 → 10개 인스턴스
  - Positive: 비율별 A+B개 단어 셔플 → 10개 인스턴스
  - FCR: 토큰 기준 위조 비율 (문서 현실)
  - IER: 인스턴스 기준 위조 비율 (모델 체감)


In [6]:
# Split 생성 함수 (위조 비율 지원 + 확장된 메타데이터)
def generate_split(name, WDICT, neg_per_writer=10, pos_per_writer=10, seed=42, pos_ratio=0.30):
    """
    베이스라인 split 생성. pos_ratio에 따라 다양한 위조 비율 세트 생성.
    L=20 기준, 확장된 메타데이터(FCR/IER) 포함.
    """
    rng = np.random.default_rng(seed)
    writer_ids = list_writer_ids(WDICT)
    bags, labels, metadata = [], [], []
    
    print(f"  {name} split 생성 중... (Writers: {len(writer_ids)}, pos_ratio={pos_ratio:.2f})")
    
    # Negative bags (단일 작성자)
    for wid in writer_ids:
        for _ in range(neg_per_writer):
            bag, metas, authors, ratio_meta = make_negative_bag(wid, WDICT[wid], rng)
            bags.append(bag)
            labels.append(0)
            
            # 확장된 메타데이터 구조
            md = {
                'authors': authors, 
                'bag_type': 'negative',
                'instances': metas,
            }
            md.update(ratio_meta)  # FCR/IER 필드들 추가
            metadata.append(md)
    
    # Positive bags (복수 작성자, 각 A당 pos_per_writer개, 파트너는 랜덤)
    for widA in writer_ids:
        for _ in range(pos_per_writer):
            widB = rng.choice([w for w in writer_ids if w != widA])
            bag, metas, authors, ratio_meta = make_positive_bag(
                widA, widB, WDICT[widA], WDICT[widB], rng, pos_ratio=pos_ratio
            )
            bags.append(bag)
            labels.append(1)
            
            # 확장된 메타데이터 구조
            md = {
                'authors': authors, 
                'bag_type': 'positive',
                'instances': metas
            }
            md.update(ratio_meta)  # FCR/IER 필드들 추가
            metadata.append(md)
    
    # 전체 셔플
    idx = rng.permutation(len(labels))
    bags = [bags[i] for i in idx]
    labels = [int(labels[i]) for i in idx]
    metadata = [metadata[i] for i in idx]
    
    # 요약 출력
    n_pos = sum(labels)
    n_tot = len(labels)
    print(f"    → Total: {n_tot}, Positive: {n_pos} ({n_pos/n_tot*100:.1f}%), Negative: {n_tot-n_pos}")
    
    # 간단 검증
    assert len(bags) == len(labels) == len(metadata), "Length mismatch"
    assert bags[0].shape == (INSTANCES_PER_BAG, WIN, embed_dim), f"Shape mismatch: {bags[0].shape}"
    
    return bags, labels, metadata

print("✓ Split 생성 함수 정의 완료 (확장된 메타데이터 지원)")

✓ Split 생성 함수 정의 완료 (확장된 메타데이터 지원)


In [7]:
# 저장 함수 (비율 태그 포함)
def save_split(bags, labels, meta, tag, pos_ratio, compat_copy=True):
    rtag = ratio_tag(pos_ratio)  # 예: '05p', '30p'
    base = os.path.join(bags_dir, f"bags_arcface_margin_{margin_value}_{rtag}_baseline_{tag}.pkl")
    with open(base, 'wb') as f:
        pickle.dump({'bags': bags, 'labels': labels, 'metadata': meta}, f)
    print(f"💾 저장: {os.path.basename(base)}")
    
    if compat_copy:
        # Stage3 호환용 별칭도 비율 태그를 포함시켜 충돌 방지
        alias = os.path.join(bags_dir, f"bags_arcface_margin_{margin_value}_{rtag}_random_{tag}.pkl")
        with open(alias, 'wb') as f:
            pickle.dump({'bags': bags, 'labels': labels, 'metadata': meta}, f)
        print(f"↪️  호환 복사: {os.path.basename(alias)}")

print("✓ 저장 함수 정의 완료 (비율 태그 지원)")

✓ 저장 함수 정의 완료 (비율 태그 지원)


In [8]:
# ==== [RUN] Generate datasets for multiple ratios ====
NEG_PW = 10
POS_PW = 10

RATIOS = [0.05, 0.10, 0.20, 0.30, 0.50]

print("🔄 Matched 모드: Train/Val/Test 모두 동일 비율")
for i, r in enumerate(RATIOS):
    print(f"\n=== [Matched] ratio={r:.2f} ({ratio_tag(r)}) ===")
    train_bags, train_labels, train_meta = generate_split('Train', train_writers, NEG_PW, POS_PW, seed=SEED_BASE+100*i+0, pos_ratio=r)
    val_bags,   val_labels,   val_meta   = generate_split('Val',   val_writers,   NEG_PW, POS_PW, seed=SEED_BASE+100*i+10, pos_ratio=r)
    test_bags,  test_labels,  test_meta  = generate_split('Test',  test_writers,  NEG_PW, POS_PW, seed=SEED_BASE+100*i+20, pos_ratio=r)

    save_split(train_bags, train_labels, train_meta, 'train', pos_ratio=r, compat_copy=True)
    save_split(val_bags,   val_labels,   val_meta,   'val',   pos_ratio=r, compat_copy=True)
    save_split(test_bags,  test_labels,  test_meta,  'test',  pos_ratio=r, compat_copy=True)

print("\n🔄 Shift 모드: Train=30% 고정, Val/Test만 비율 변경")
r_train = 0.30
print(f"\n=== [Shift] Train ratio={r_train:.2f} ({ratio_tag(r_train)}) ===")
train_bags_30, train_labels_30, train_meta_30 = generate_split('Train', train_writers, NEG_PW, POS_PW, seed=SEED_BASE+999, pos_ratio=r_train)
val_bags_30,   val_labels_30,   val_meta_30   = generate_split('Val',   val_writers,   NEG_PW, POS_PW, seed=SEED_BASE+1009, pos_ratio=r_train)
# 학습용 세트 저장
save_split(train_bags_30, train_labels_30, train_meta_30, 'train_shiftbase', pos_ratio=r_train, compat_copy=True)
save_split(val_bags_30,   val_labels_30,   val_meta_30,   'val_shiftbase',   pos_ratio=r_train, compat_copy=True)

# 평가용(다양한 비율) Val/Test만 추가 생성
for i, r in enumerate(RATIOS):
    print(f"\n--- [Shift] Eval-only ratio={r:.2f} ({ratio_tag(r)}) ---")
    val_bags_s,  val_labels_s,  val_meta_s  = generate_split('Val',  val_writers,  NEG_PW, POS_PW, seed=SEED_BASE+2000+10*i, pos_ratio=r)
    test_bags_s, test_labels_s, test_meta_s = generate_split('Test', test_writers, NEG_PW, POS_PW, seed=SEED_BASE+2000+20*i, pos_ratio=r)
    save_split(val_bags_s,  val_labels_s,  val_meta_s,  f'val_shift_{ratio_tag(r)}',  pos_ratio=r, compat_copy=True)
    save_split(test_bags_s, test_labels_s, test_meta_s, f'test_shift_{ratio_tag(r)}', pos_ratio=r, compat_copy=True)

🔄 Matched 모드: Train/Val/Test 모두 동일 비율

=== [Matched] ratio=0.05 (05p) ===
  Train split 생성 중... (Writers: 180, pos_ratio=0.05)
    → Total: 3600, Positive: 1800 (50.0%), Negative: 1800
  Val split 생성 중... (Writers: 60, pos_ratio=0.05)
    → Total: 1200, Positive: 600 (50.0%), Negative: 600
  Test split 생성 중... (Writers: 60, pos_ratio=0.05)
    → Total: 1200, Positive: 600 (50.0%), Negative: 600
💾 저장: bags_arcface_margin_0.4_05p_baseline_train.pkl
↪️  호환 복사: bags_arcface_margin_0.4_05p_random_train.pkl
💾 저장: bags_arcface_margin_0.4_05p_baseline_val.pkl
↪️  호환 복사: bags_arcface_margin_0.4_05p_random_val.pkl
💾 저장: bags_arcface_margin_0.4_05p_baseline_test.pkl
↪️  호환 복사: bags_arcface_margin_0.4_05p_random_test.pkl

=== [Matched] ratio=0.10 (10p) ===
  Train split 생성 중... (Writers: 180, pos_ratio=0.10)
    → Total: 3600, Positive: 1800 (50.0%), Negative: 1800
  Val split 생성 중... (Writers: 60, pos_ratio=0.10)
    → Total: 1200, Positive: 600 (50.0%), Negative: 600
  Test split 생성 중... (Writer

In [9]:
# 생성된 데이터 검증 및 샘플 확인 (L=20 기준, FCR/IER 분석)
print("🔍 생성된 데이터 검증 (L=20 기준):")

# 각 비율별 샘플 메타데이터 확인
sample_ratios = [0.05, 0.30, 0.50]  # 극단 비율들 확인

for r in sample_ratios:
    print(f"\n📋 {ratio_tag(r)} 비율 샘플 메타데이터:")
    
    # 해당 비율로 생성된 작은 샘플 생성 (검증용)
    test_bags_sample, test_labels_sample, test_meta_sample = generate_split(
        'Test_Sample', test_writers, 1, 1, seed=SEED_BASE+9999, pos_ratio=r
    )
    
    # Negative bag 샘플
    neg_sample = next(meta for meta, label in zip(test_meta_sample, test_labels_sample) if label == 0)
    print(f"  Negative bag:")
    print(f"    - Authors: {neg_sample['authors']} (개수: {len(neg_sample['authors'])})")
    print(f"    - Type: {neg_sample['bag_type']}")
    print(f"    - FCR target/realized: {neg_sample['FCR_target']:.2f}/{neg_sample['FCR_realized']:.2f}")
    print(f"    - IER expected/realized: {neg_sample['IER_expected_allK']:.2f}/{neg_sample['IER_realized_usedK']:.2f}")
    print(f"    - Tokens A/B: {neg_sample['tokens_A']}/{neg_sample['tokens_B']}")
    print(f"    - L/W/K: {neg_sample['L']}/{neg_sample['W']}/{neg_sample['K_total']}")
    
    # Positive bag 샘플
    pos_sample = next(meta for meta, label in zip(test_meta_sample, test_labels_sample) if label == 1)
    print(f"  Positive bag:")
    print(f"    - Authors: {pos_sample['authors']} (개수: {len(pos_sample['authors'])})")
    print(f"    - Type: {pos_sample['bag_type']}")
    print(f"    - FCR target/realized: {pos_sample['FCR_target']:.2f}/{pos_sample['FCR_realized']:.2f}")
    print(f"    - IER expected/realized: {pos_sample['IER_expected_allK']:.2f}/{pos_sample['IER_realized_usedK']:.2f}")
    print(f"    - Tokens A/B: {pos_sample['tokens_A']}/{pos_sample['tokens_B']}")
    print(f"    - B writer: {pos_sample['b_writer']}")
    print(f"    - L/W/K: {pos_sample['L']}/{pos_sample['W']}/{pos_sample['K_total']}")
    print(f"    - 첫 인스턴스 작성자들: {pos_sample['instances'][0]['writer_ids']}")
    
    # FCR 정확성 검증
    expected_a, expected_b = ratio_to_counts_exact(r)
    print(f"    - 기대 토큰 A/B: {expected_a}/{expected_b} (비율: {expected_b/(expected_a+expected_b):.2f})")

# 비율별 기대 IER 표 출력
print(f"\n📊 비율별 기대값 표 (L={TOTAL_TOKENS_POS}, W={WIN}):")
print("┌─────────┬──────────┬──────────┬────────────┬────────────┐")
print("│ 비율(%) │  A 토큰  │  B 토큰  │ FCR(토큰)  │ IER(기대)  │")
print("├─────────┼──────────┼──────────┼────────────┼────────────┤")
for r in [0.05, 0.10, 0.20, 0.30, 0.50]:
    a, b = ratio_to_counts_exact(r)
    fcr = b / (a + b)
    ier = expected_ier(TOTAL_TOKENS_POS, WIN, b)
    print(f"│  {r*100:5.0f}%  │    {a:2d}    │    {b:2d}    │   {fcr:6.2f}   │   {ier:6.2f}   │")
print("└─────────┴──────────┴──────────┴────────────┴────────────┘")

print(f"\n✅ 모든 검증 통과 - L={TOTAL_TOKENS_POS} 기준 위조 비율별 데이터 준비 완료!")
print(f"🚀 Stage 3에서 다음과 같이 활용:")
print(f"  - FCR 분석: 문서 현실 위조 비율 (토큰 기준)")
print(f"  - IER 분석: 모델 체감 위조 비율 (인스턴스 기준)")
print(f"  - Matched 모드: 각 비율별 독립 학습 {len([0.05, 0.10, 0.20, 0.30, 0.50])}회")
print(f"  - Shift 모드: 30% 학습 → 5/10/20/30/50% 평가")

🔍 생성된 데이터 검증 (L=20 기준):

📋 05p 비율 샘플 메타데이터:
  Test_Sample split 생성 중... (Writers: 60, pos_ratio=0.05)
    → Total: 120, Positive: 60 (50.0%), Negative: 60
  Negative bag:
    - Authors: [250] (개수: 1)
    - Type: negative
    - FCR target/realized: 0.00/0.00
    - IER expected/realized: 0.00/0.00
    - Tokens A/B: 20/0
    - L/W/K: 20/5/16
  Positive bag:
    - Authors: [261, 297] (개수: 2)
    - Type: positive
    - FCR target/realized: 0.05/0.05
    - IER expected/realized: 0.25/0.20
    - Tokens A/B: 19/1
    - B writer: 297
    - L/W/K: 20/5/16
    - 첫 인스턴스 작성자들: [261, 297, 261, 261, 261]
    - 기대 토큰 A/B: 19/1 (비율: 0.05)

📋 30p 비율 샘플 메타데이터:
  Test_Sample split 생성 중... (Writers: 60, pos_ratio=0.30)
    → Total: 120, Positive: 60 (50.0%), Negative: 60
  Negative bag:
    - Authors: [250] (개수: 1)
    - Type: negative
    - FCR target/realized: 0.00/0.00
    - IER expected/realized: 0.00/0.00
    - Tokens A/B: 20/0
    - L/W/K: 20/5/16
  Positive bag:
    - Authors: [261, 297] (개수: 2)
   

In [10]:
# 최종 요약 (L=20 기준, FCR/IER 분석 포함)
print("=" * 80)
print("🎯 Stage 2 위조 비율 실험 완료! (L=20 기준)")
print("=" * 80)
print("✅ 생성된 데이터셋:")

print("\n📊 Matched 모드 (각 비율별 독립):")
RATIOS = [0.05, 0.10, 0.20, 0.30, 0.50]
for r in RATIOS:
    rtag = ratio_tag(r)
    expected_a, expected_b = ratio_to_counts_exact(r)
    ier_exp = expected_ier(TOTAL_TOKENS_POS, WIN, expected_b)
    print(f"  • {rtag} ({r:.0%}): A={expected_a}, B={expected_b} 토큰")
    print(f"    - FCR(토큰): {expected_b/(expected_a+expected_b):.2f}, IER(기대): {ier_exp:.2f}")
    print(f"    - bags_arcface_margin_{margin_value}_{rtag}_baseline_*.pkl")
    print(f"    - bags_arcface_margin_{margin_value}_{rtag}_random_*.pkl (호환)")

print("\n📊 Shift 모드 (30% 학습, 다양한 평가):")
r_train = 0.30
expected_a_train, expected_b_train = ratio_to_counts_exact(r_train)
expected_ier_train = expected_ier(TOTAL_TOKENS_POS, WIN, expected_b_train)
print(f"  • 학습용: 30p (30%) - A={expected_a_train}, B={expected_b_train} 토큰")
print(f"    - FCR: {expected_b_train/(expected_a_train+expected_b_train):.2f}, IER(기대): {expected_ier_train:.2f}")
print(f"    - bags_arcface_margin_{margin_value}_30p_baseline_train_shiftbase.pkl")
print(f"    - bags_arcface_margin_{margin_value}_30p_random_train_shiftbase.pkl")
print(f"    - bags_arcface_margin_{margin_value}_30p_baseline_val_shiftbase.pkl")
print(f"    - bags_arcface_margin_{margin_value}_30p_random_val_shiftbase.pkl")
print(f"  • 평가용:")
for r in RATIOS:
    rtag = ratio_tag(r)
    expected_a, expected_b = ratio_to_counts_exact(r)
    ier_exp = expected_ier(TOTAL_TOKENS_POS, WIN, expected_b)
    print(f"    - {rtag}_shift ({r:.0%}): A={expected_a}, B={expected_b}, FCR={expected_b/(expected_a+expected_b):.2f}, IER={ier_exp:.2f}")

print(f"\n🔬 향상된 분석 기능:")
print(f"  • FCR (Forgery Content Ratio): 토큰 기준 위조 비율 - 문서의 현실적 위조 정도")
print(f"  • IER (Instance Exposure Ratio): 인스턴스 기준 위조 비율 - 모델이 체감하는 위조 정도")
print(f"  • 각 bag 메타데이터에 FCR_target/realized, IER_expected/realized 모두 기록")
print(f"  • b_writer 정보로 어떤 작성자가 B(위조자)인지 추적 가능")

print(f"\n🎯 연구 목적 (L={TOTAL_TOKENS_POS} 기준):")
print(f"  • 낮은 위조 비율(5%, 10%)에서 모델 성능 평가")
print(f"  • FCR vs IER 차이 분석으로 모델 학습 효과 이해")
print(f"  • Recall 유지 능력 측정 (위조 놓치지 않기)")
print(f"  • 도메인 적응 능력 평가 (Shift 모드)")

print(f"\n🚀 다음 단계: Stage 3에서 AB-MIL 학습 및 FCR/IER 기반 성능 분석")
print(f"  • Matched 분석: 각 위조 비율별 최적 성능 측정")
print(f"  • Shift 분석: 30% 학습 모델의 다양한 비율 일반화 능력")
print(f"  • FCR/IER 상관관계: 토큰 vs 인스턴스 위조 비율의 모델 성능 영향")

🎯 Stage 2 위조 비율 실험 완료! (L=20 기준)
✅ 생성된 데이터셋:

📊 Matched 모드 (각 비율별 독립):
  • 05p (5%): A=19, B=1 토큰
    - FCR(토큰): 0.05, IER(기대): 0.25
    - bags_arcface_margin_0.4_05p_baseline_*.pkl
    - bags_arcface_margin_0.4_05p_random_*.pkl (호환)
  • 10p (10%): A=18, B=2 토큰
    - FCR(토큰): 0.10, IER(기대): 0.45
    - bags_arcface_margin_0.4_10p_baseline_*.pkl
    - bags_arcface_margin_0.4_10p_random_*.pkl (호환)
  • 20p (20%): A=16, B=4 토큰
    - FCR(토큰): 0.20, IER(기대): 0.72
    - bags_arcface_margin_0.4_20p_baseline_*.pkl
    - bags_arcface_margin_0.4_20p_random_*.pkl (호환)
  • 30p (30%): A=14, B=6 토큰
    - FCR(토큰): 0.30, IER(기대): 0.87
    - bags_arcface_margin_0.4_30p_baseline_*.pkl
    - bags_arcface_margin_0.4_30p_random_*.pkl (호환)
  • 50p (50%): A=10, B=10 토큰
    - FCR(토큰): 0.50, IER(기대): 0.98
    - bags_arcface_margin_0.4_50p_baseline_*.pkl
    - bags_arcface_margin_0.4_50p_random_*.pkl (호환)

📊 Shift 모드 (30% 학습, 다양한 평가):
  • 학습용: 30p (30%) - A=14, B=6 토큰
    - FCR: 0.30, IER(기대): 0.87
    - bags_arc